In [62]:
import itertools
import json
import pickle
import re

import numpy as np
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

from tqdm.notebook import tqdm
tqdm.pandas()

In [63]:
with open("../../data/pseudowords/CoMaPP_all.json") as json_file:
    data = json.load(json_file)
    
# d+1 since we are not replacing the ke-lex this time!
data = [{"example": d["target1"], "cue": " ".join(d["target1"].split()[:d["query_idx"]+1]), "pseudoword": d["label"]} for d in data]
df = pd.DataFrame.from_dict(data).drop_duplicates(ignore_index=True)
df

,example,cue,pseudoword
0,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
1,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
2,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
3,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
4,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
12128,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
12129,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
12130,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99
12131,"An seiner Hand geht ein Mädchen , einen Kopf k...","An seiner Hand geht ein Mädchen , einen Kopf k...",er99


In [64]:
df['index'] = df['pseudoword'].str.extract('(\d+)').astype(int)
df.set_index('index', inplace=True)

df

,example,cue,pseudoword
index,,,
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99


In [65]:
contextleft = pd.read_pickle("../../data/pseudowords/contextleft_text.pickle")

def update_cue(row):
    output = row[['example', 'cue']]
    if row['cue'] == '':  # if the string in cue is empty
        # match the index of row with contextleft['construction_id'] and match contextleft['text'] with row['example'] and create matching_entry
        matching_entry = contextleft.loc[(contextleft['construction_id'] == row.name) & (contextleft['text'] == row['example']), 'contextleft'].tolist()
        if len(matching_entry) > 0:
            output = [matching_entry[0] + " " + row['example'], matching_entry[0]]
    return output

# Add the left context if there is no cue up until the pseudoword.
df[["example", "cue"]] = df.apply(update_cue, axis=1)
df

,example,cue,pseudoword
index,,,
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99


In [66]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'construction'}, inplace=True)

result_df = df.groupby(['construction', 'pseudoword']).agg({'example': list, 'cue': list})

result_df

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  
construction pseudoword                                                     
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...  
             erst5       [Trainer Lucien Favre hatte schon seine beiden...  
             gar5        [Es hat Afghanistan nicht stabilisiert, Es hat...  
             nicht5      [Es, Es hat Afghanistan, Dass, Dass es in der ...  
             recht5      [Trainer Lucien Favre hatte schon seine beiden...  
...                                                                    ...  
1884         Gold1884    [Schweigen ist Silber , reden ist, Schweigen i...  
             Silber1884  [Schweigen ist, Schweigen ist, Angeben ist, Sc...  
             ist1884     [Schweigen, Schweigen, Angeben, Schweigen, " Q...  
1986         kaum1986    [Die Vorhut vor 20.000 Jahren, Die Vorhut vor ...  
1987         sehr1987    [Solche Menschen können, Solche Menschen könne...  

[562 rows x 2 columns]

In [67]:
with open("../../out/definitions.pickle", "rb") as definitions_file:
    definitions = pd.DataFrame.from_dict(pickle.load(definitions_file), orient="index", columns=["definition"])
    
definitions

,definition
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio..."
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de..."
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru..."
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""..."
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ..."
...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g..."
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e..."
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han..."
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z..."


In [68]:
examples = pd.merge(result_df, definitions, how="inner", left_on="construction", right_index=True)
examples

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert, Es hat...   
             nicht5      [Es, Es hat Afghanistan, Dass, Dass es in der ...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist, Schweigen i...   
             Silber1884  [Schweigen ist, Schweigen ist, Angeben ist, Sc...   
             ist1884     [Schweigen, Schweigen, Angeben, Schweigen, " Q...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren, Die Vorhut vor ...   
1987         sehr1987    [Solche Menschen können, Solche Menschen könne...   

                                                                definition  
construction pseudoword                                                     
5            Und5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             erst5       Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             gar5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             nicht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             recht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
...                                                                    ...  
1884         Gold1884    Die "Intensivierung_Komparativ:Xist_SilberYist...  
             Silber1884  Die "Intensivierung_Komparativ:Xist_SilberYist...  
             ist1884     Die "Intensivierung_Komparativ:Xist_SilberYist...  
1986         kaum1986    Die Konstruktion "Relativierung:kaumADJ" gehör...  
1987         sehr1987    Die Konstruktion "Intensivierung:sehrV" gehört...  

[562 rows x 3 columns]

##### Generieren neuer Sätze:

Load the vanilla mbart model:

In [72]:
model = MBartForConditionalGeneration.from_pretrained(
    "facebook/mbart-large-50", return_dict=True
) 
tokenizer = MBart50TokenizerFast.from_pretrained(
    "facebook/mbart-large-50", src_lang="de_DE", tgt_lang="de_DE"
)
model.model.encoder.embed_tokens

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(30000, 768, padding_idx=0)

Complete the cues:

In [22]:
def complete_cues(row):
    try:
        output_texts = []
        scores = []
        for cue, example in zip(row["cue"], row["example"]):
            input_text = cue
            
            # print(tokenizer.tokenize(input_text))
            
            target_length = int(1.5 * len(example))  # allow double the length of the original sentence
            
            outputs = tokenizer(input_text, return_tensors="pt")
            outputs = model.generate(outputs["input_ids"], max_length=target_length, num_return_sequences=1,
                                     num_beams=10, output_scores=True, return_dict_in_generate=True)
            output_text = tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)
            score = torch.exp(outputs.sequences_scores)
            output_texts += output_text
            
            scores.append(score)
        print(row["pseudoword"], str(output_texts), str([float(score) for score in scores]))
        
        return pd.DataFrame([str(output_texts), str([float(score) for score in scores])])
    except Exception as e:
        print(".", end="")
        return pd.DataFrame([row["pseudoword"], str(e), "[-1.0]"])

examples_reset = examples.reset_index()
pseudoword_output_scores = examples_reset[["example", "cue", "pseudoword"]].progress_apply(complete_cues, axis=1)
pseudoword_output_scores

  0%|          | 0/562 [00:00<?, ?it/s]

...


KeyboardInterrupt



In [ ]:
examples = pseudoword_output_scores[["pseudoword", "generated", "score"]]

examples

#### Anzahl neuer Sätze

In [ ]:
examples.to_csv(f"../../out/comapp/data_mbart_vanilla.tsv", sep="\t", decimal=",")
examples.to_excel(f"../../out/comapp/data_mbart_vanilla.xlsx")

In [ ]:
pseudoword_output_scores.to_csv("../../out/comapp/data_mbart_vanilla_complete.tsv", sep="\t", decimal=",")